# Drivers's performance - Braspress

## Libraries

In [25]:
import numpy as np
import pandas as pd

## Settings

In [ ]:
settings = {
    "telemetry_dir": "Dataset/Telemetry Data",
    "braspress_branches_path": "Dataset/braspress_branches.csv",
    "dates": (
        (2024, 10),
        (2024, 11),
        (2024, 12)
    ),
    "min_displacement": 5.0
}

## Read Telemetry Data

### Auxiliary Methods

In [154]:
def clean_telemetry_data(data: pd.DataFrame) -> pd.DataFrame:
    # Rename columns
    columns_rename = {        
        "Frota": "fleet",
        "CPF": "driver_id",
        "Lat/Long inicial": "initial_coord",
        "Lat/Long final": "final_coord",
        "Distância (Km)": "distance",
        "Total Litros Consumido": "fuel"
    }
    clean_data = data[columns_rename.keys()].astype(str).rename(columns=columns_rename)

    # Convert brazilian decimal to Python float
    clean_data["distance"] = clean_data["distance"].apply(lambda d: d.replace(",", ".")).astype(float)
    clean_data["fuel"] = clean_data["fuel"].apply(lambda d: d.replace(",", ".")).astype(float)
    
    # Split INITIAL coordinates into two columns: latitude and longitude
    clean_data[["initial_latitude", "initial_longitude"]] = clean_data["initial_coord"].str.split(', ', n=1, expand=True)
    clean_data["initial_latitude"] = clean_data["initial_latitude"].str.replace(',', '.', 1).astype(float)
    clean_data["initial_longitude"] = clean_data["initial_longitude"].str.replace(',', '.', 1).astype(float)
    
    # Split FINAL coordinates into two columns: latitude and longitude
    clean_data[["final_latitude", "final_longitude"]] = clean_data["final_coord"].str.split(', ', n=1, expand=True)
    clean_data["final_latitude"] = clean_data["final_latitude"].str.replace(',', '.', 1).astype(float)
    clean_data["final_longitude"] = clean_data["final_longitude"].str.replace(',', '.', 1).astype(float)

    # Remove initial and final coordinates after creating latitude and longitude columns
    clean_data.drop(columns=["initial_coord", "final_coord"], inplace=True)

    # Remove non-defined drivers
    clean_data = clean_data[clean_data["driver_id"] != '-']

    # Remove small distances
    clean_data = clean_data[clean_data["distance"] >= settings["min_displacement"]]
    
    return clean_data

def get_telemetry_data(year: int, month: int) -> pd.DataFrame:
    path: str = f"{settings["telemetry_dir"]}/{year:04}-{month:02}.csv"
    raw_df: pd.DataFrame = pd.read_csv(path, delimiter=";", dtype = str)
    
    return raw_df

def get_braspress_branches() -> pd.DataFrame:
    raw_df: pd.DataFrame = (pd
                            .read_csv(settings["braspress_branches_path"], delimiter=";", dtype = str)
                            .set_index("id"))
    
    return raw_df

**Analysis of 2024-10 data**

Extract data

In [151]:
raw_oct2024: pd.DataFrame = get_telemetry_data(2024, 10)

In [155]:
braspress_branches = get_braspress_branches()
braspress_branches

,latitude,longitude
id,,
IRJ,-22.80118791995254,-43.347953176084495
XAP,-27.00098702010198,-52.652606771840524
SDR,-22.45676362769558,-44.41378638336371
VOT,-20.4511947612515,-49.89090467612616
VIX,-20.159280790468877,-40.23974523295941


In [141]:
clean_oct2024 = clean_telemetry_data(raw_oct2024)

In [142]:
clean_oct2024

,fleet,driver_id,distance,fuel,initial_latitude,initial_longitude,final_latitude,final_longitude
9,281,3347814770,391.62,121.63,-22.800595,-43.348415,-23.456463,-46.483862
128,2559,13312035899,139.83,48.32,-23.456183,-46.483673,-23.293822,-47.718856
129,2559,13312035899,280.48,106.23,-23.293822,-47.718856,-21.864598,-49.573604
130,2559,13312035899,260.55,92.40,-21.864598,-49.573604,-20.781739,-51.659293
194,21374,1537127756,111.59,34.35,-19.969836,-44.068936,-20.539833,-44.642003
...,...,...,...,...,...,...,...,...
121383,21712,86276096598,42.94,10.52,-12.283806,-38.921840,-12.278379,-38.924826
121385,21712,86276096598,12.37,2.77,-12.277831,-38.924438,-12.324238,-38.868015
121389,21712,86276096598,18.73,4.76,-12.331394,-38.858686,-12.308026,-38.881907
121390,21712,86276096598,14.67,4.30,-12.308025,-38.881869,-12.289710,-38.913148
